# MLP

In [ ]:


import pandas as pd 
import numpy as np 
import sklearn as ski

from sklearn.model_selection import KFold, train_test_split
from sklearn.neural_network import MLPClassifier as multilp
from sklearn.metrics import confusion_matrix

df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                 'machine-learning-databases/wine/wine.data', names=(
                                            'Class', 
                                            'Alcohol', 
                                            'Malic acid',
                                            'Ash', 
                                            'Alcalinity of ash',
                                            'Magnesium', 
                                            'Total phenols',
                                            'Flavanoids', 
                                            'Nonflavanoid phenols',
                                            'Proanthocyanins', 
                                            'Color intensity',
                                            'Hue', 
                                            'OD280/OD315 of diluted wines',
                                            'Proline' ))
#training_num =int( len(df) * .6 )
#classif = df.iloc[1:training_num,0]
#x_train = df.loc[1:training_num,df.columns != 'Class']
#classif
classif = df['Class']
df = df.drop(['Class'],axis = 1)

#for col in range(len(df.columns)):
 # sum = df.iloc[:,col].sum()
  #print(sum)
  #for row in range(len(df)):
   # df.iloc[row,col] = df.iloc[row,col] / sum 

for col in range(len(df.columns)):
  mean = df.iloc[:,col].sum()/len(df)
  std = df.iloc[:,col].std()
  for row in range(len(df)):
   df.iloc[row,col] = (df.iloc[row,col] - mean) / std
df['Class'] = classif
np.random.seed(99)
x_train, x_test, y_train, y_test = train_test_split(df, df['Class'], test_size=0.2)
learning_rate = .2
#for i in range(len(x_train)):
 # print(y_train.iloc[i])
# Dropping targets from training set, we do not want to train over targets
#print(np.shape(x_train),np.shape(x_test),np.shape(y_train),np.shape(y_test))
x_train = x_train.drop(['Class'], axis=1)
x_test = x_test.drop(['Class'], axis=1)




In [ ]:

# Number of hidden neurons is 6 so # of weights from all input nodes to hidden neurons is 13 * 6 
# Number of output neurons = 3, 1 for each class 
from numpy.random import default_rng

np.random.seed(775) 
#100 good
#71 even better
#775 amazing 
output = [0,0,0]

class HiddenLayer():
  hiddenNodes = []
  bias = -1
  def __init__(self,numOfHN):
    i = 0 
    while(i < numOfHN):
        hn = HiddenNode() 
        HiddenLayer.hiddenNodes.append(hn)
        i+=1
    

class HiddenNode:
  seed = 0 
  seed1 = 400 
  pre_af_val = 0 
  post_af_val = 0 
  inp_to_h_weights = []
  h_to_out = []
  def __init__(self):
    self.inp_to_h_weights =  np.random.uniform(-1,1,size = (1,13))	#default_rng(HiddenNode.seed).random((13)) #numpy.random.uniform()	
    self.inp_to_h_weights = self.inp_to_h_weights[0][:]
    HiddenNode.seed += 1 
    self.h_to_out = np.random.uniform(-1,1,size = (1,3)) #default_rng(HiddenNode.seed1).random(3)
    self.h_to_out = self.h_to_out[0][:]
    HiddenNode.seed1 += 20
    #print(self.inp_to_h_weights)
    #print(len(self.inp_to_h_weights))
    print(self.h_to_out)


hl = HiddenLayer(6) #initialize MLP with 1 hidden layer of 6 nodes, randomize input to hidden layer weights (13 weights per hidden neuron), 
                    #randomize hidden to output weights (3 weights per hidden)


[-0.14139123 -0.68429059 -0.33354377]
[-0.59525376 -0.09417655  0.25755592]
[ 0.58984275  0.04925569 -0.88868765]
[-0.95431318 -0.01769389  0.44438857]
[ 0.999286   -0.85877564 -0.15392371]
[ 0.14141774 -0.38462306  0.95804651]


In [ ]:

#for each row in training dataframe 
#     feed forward 
#     calculate error 
#     backpropagate 

for row in range(len(x_train)) : #each row used as training data, fed forward and backpropagation for each row 
  #multiply each col by corresponding weight 
  for col in range(len(x_train.columns)): #iterate through columns
    for hn in range(len(hl.hiddenNodes)) :#multiply the column value by each weight that is attached to the input node that corresponds to the column 
                                          #add that value to the pre_af_val for each hidden layer node 
      hl.hiddenNodes[hn].pre_af_val  +=   (x_train.iloc[row,col] * hl.hiddenNodes[hn].inp_to_h_weights[col])
  
  #after going through all the columns and calculating pre-activation function value, we use activation function on pre-activation function value to get post-actfunc val 
  for hn in range(len(hl.hiddenNodes)):
    #print(hl.hiddenNodes[hn].pre_af_val)
    hl.hiddenNodes[hn].post_af_val = 1/(1 + np.exp(-(hl.hiddenNodes[hn].pre_af_val + hl.bias)))
    #print("Post Act Func Val: {post_af}".format(post_af = hl.hiddenNodes[hn].post_af_val))

  #loop through hidden neurons and multiply post-af val by corresponding weights to get output pre-af values 
  for hn in range(len(hl.hiddenNodes)):
    for o in range(len(output)):
      output[o] += (hl.hiddenNodes[hn].h_to_out[o] * hl.hiddenNodes[hn].post_af_val)
      
  sum1 = 0 
  for o in range(len(output)):
    output[o] = np.exp(output[o]) 
    sum1 += output[o]

  for o in range(len(output)):
    output[o] = output[o]/sum1


  #for i in range(len(output)):
    #print("Output {i}: {output}".format(i = i, output = output[i])) 
  #print() 

  #compute the loss function 
  sse = 0 
  t_output = [0,0,0]
  t_output[y_train.iloc[row] - 1] = 1 #create the target output vector 
  #if(y_train[row] == 1):
   # t_output[0] = 1
  #elif(y_train[row] == 2):
   # t_output[1] = 1
  #else:
   # t_output[2] = 1 

  for o in range(len(output)):
    sse += (output[o] - t_output[o])** 2 
  sse = sse * .5 

  #calculate gradient for 2nd layer weights
  #gradient is different for each output neuron 

  grad_out = [0,0,0]
  for o in range(len(output)):
    grad_out[o] = (output[o] - t_output[o]) * output[o] * (1 - output[o])
  count = 0 
  for hn in range(len(hl.hiddenNodes)):
    for i in range(len(hl.hiddenNodes[hn].h_to_out)): 
      count+= 1 
      #print("Iteration: {number}".format(number = count))
      if(i < 3):
        grad_k = grad_out[0]
      elif(i < 6):
        grad_k = grad_out[1]
      else:
        grad_k = grad_out[2]
      #print("Weight before change: {weight}".format(weight = hl.hiddenNodes[hn].h_to_out[i]))
      hl.hiddenNodes[hn].h_to_out[i] = hl.hiddenNodes[hn].h_to_out[i] - (learning_rate * grad_k * hl.hiddenNodes[hn].post_af_val)
      #print("Weight after change: {weight}".format(weight = hl.hiddenNodes[hn].h_to_out[i]))
      #print()

  #calculate gradient for each hidden layer neuron 
  grad_h = [0,0,0,0,0,0]
  sum_g_weights = [0,0,0,0,0,0]
  #need summation of gradient times h_to_output
  for hn in range(len(hl.hiddenNodes)):
    for w in range(len(hl.hiddenNodes[hn].h_to_out)):
      g = 0 
      if(w == 0):
        g = grad_out[0]
      elif(w == 1):
        g = grad_out[1]
      else:
        g = grad_out[2]
      sum_g_weights[hn] += (hl.hiddenNodes[hn].h_to_out[w] * g)


  for g in range(len(grad_h)):
    grad_h[g] = hl.hiddenNodes[g].post_af_val*(1 - hl.hiddenNodes[g].post_af_val) * sum_g_weights[g]
  for hn in range(len(hl.hiddenNodes)):
      for w in range(len(hl.hiddenNodes[hn].inp_to_h_weights)):
        hl.hiddenNodes[hn].inp_to_h_weights[w] = hl.hiddenNodes[hn].inp_to_h_weights[w] - learning_rate * grad_h[hn]


  #after completing backpropagation, pre and post activation function value and output needs to be reset to zero 
  for hn in range(len(hl.hiddenNodes)):
    hl.hiddenNodes[hn].pre_af_val = 0 
    hl.hiddenNodes[hn].post_af_val = 0 
  #output also needs to be reset 
  for i in range(len(output)):
    output[i] = 0 

In [ ]:
correct_count = 0 
for row in range(len(x_test)):
  for col in range(len(x_test.columns)): #iterate through columns
    for hn in range(len(hl.hiddenNodes)) :#multiply the column value by each weight that is attached to the input node that corresponds to the column 
                                          #add that value to the pre_af_val for each hidden layer node 
      hl.hiddenNodes[hn].pre_af_val  +=   (x_test.iloc[row,col] * hl.hiddenNodes[hn].inp_to_h_weights[col])
  
  #after going through all the columns and calculating pre-activation function value, we use activation function on pre-activation function value to get post-actfunc val 
  for hn in range(len(hl.hiddenNodes)):
    #print(hl.hiddenNodes[hn].pre_af_val)
    hl.hiddenNodes[hn].post_af_val = 1/(1 + np.exp(-(hl.hiddenNodes[hn].pre_af_val + hl.bias)))
    #print("Post Act Func Val: {post_af}".format(post_af = hl.hiddenNodes[hn].post_af_val))

  #loop through hidden neurons and multiply post-af val by corresponding weights to get output pre-af values 
  for hn in range(len(hl.hiddenNodes)):
    for o in range(len(output)):
      output[o] += (hl.hiddenNodes[hn].h_to_out[o] * hl.hiddenNodes[hn].post_af_val)
      
  sum1 = 0 
  #use softmax activation function to get which class
  for o in range(len(output)):
    output[o] = np.exp(output[o]) 
    sum1 += output[o]

  for o in range(len(output)):
    output[o] = output[o]/sum1
  ####################################
  #print(output)
  target = y_test.iloc[row]
  maxProb = 0 
  class_output = 0 
  for i in range(len(output)):
    if(output[i] > maxProb):
      maxProb = output[i]
      class_output = i + 1 
  #print("Target: " + str(target))
  if(target == class_output):
    #print("Correct")
    correct_count+=1
  
  #print()
  # Activation function values need to be reset and output needs to be reset
  for hn in range(len(hl.hiddenNodes)):
    hl.hiddenNodes[hn].pre_af_val = 0 
    hl.hiddenNodes[hn].post_af_val = 0 
  #output also needs to be reset 
  for i in range(len(output)):
    output[i] = 0 

print("Correct percentage: " + str(correct_count / len(x_test)))

Correct percentage: 0.6666666666666666
